Using any OpenAI model or Groq (llama 70b) model, solve the aisle mapping problem.  OpenAI code is provided here. You will do the usual 3 steps in terms of mounting the Google drive, your API key and install one of the LLM models, and import pandas.  Your goal is to do a model comparison and validation.
**The code below works for OpenAI gpt-4o model.  Have not tested it on Llama model.  Also, this code with the dataset  may blow your budget if you are not careful with the size of test dataset, so exercise caution.**

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!pip install -r "/content/drive/MyDrive/LLM/Groq/requirements.txt"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.0/974.0 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.7/314.7 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.1
    Uninstalling packaging-24.1:
      Successfully uninstalled packaging-24.1


In [ ]:
import os  ## step-3
# Read and set the environment variable from the .bashrc file
with open('/content/drive/MyDrive/LLM/Groq/.bashrc') as file:
    for line in file:
        if line.startswith('export '):
            var, value = line[len('export '):].strip().split('=')
            os.environ[var] = value

# Verify that the environment variable is set
!echo $GROQ_API_KEY



import os  ## step-3
# Read and set the environment variable from the .bashrc file
with open('/content/drive/MyDrive/LLM/.bashrc') as file:
    for line in file:
        if line.startswith('export '):
            var, value = line[len('export '):].strip().split('=')
            os.environ[var] = value

# Verify that the environment variable is set
!echo $OPENAI_API_KEY

In [16]:

from groq import Groq
client = Groq(api_key = os.getenv('GROQ_API_KEY'))

In [17]:
import pandas as pd
df = pd.read_excel('Aisle-Mapping.xlsx')

In [18]:
groceries = df['Grocery ITEM'].dropna().tolist()
aisles = df['Aisle Category'].dropna().tolist()

In [19]:
batch_size = 50  # Adjust the batch size as needed
separator=","

# Function to get the best matches for a batch of keywords using chat completion
def get_best_matches_batch(grocery_batch, aisle_list):
    prompt = "Match each grocery item in the  grocery list with the most appropriate aisle category from the provided list of aisle categories.\n\n"
    prompt += "The grocery list items are separated by commas. The list of aisles are also separated by commas. \n\n"
    prompt += "List of grocery items to match:\n" + "\n"+ "\n"+separator.join(grocery_batch)+  "\n\n"
    prompt += "List of provided aisle categories:\n" + "\n"+ "\n"+separator.join(aisle_list)+ "\n\n"
    prompt += "If an appropriate aisle category is not to be found in the list, use Other \n\n"
    prompt += "Return the matches in the JSON format 'grocery item' : 'aisle category' \n\n"
    prompt+="You must absolutely make sure that each grocery item is mapped to an aisle category"
    #print(prompt)
    response = client.chat.completions.create(
        model="mixtral-8x7b-32768",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that matches grocery items to aisle categories based on a typical grocery store or a supermarket in the USA."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=4096,
        temperature=0.0
    )
    matches = response.choices[0].message.content
    return matches

In [20]:
grocery_batch = groceries[0: batch_size]
batch_matches = get_best_matches_batch(grocery_batch, aisles)
print(batch_matches)




{
"tropical fruit": "Produce",
"whole milk": "Dairy",
"pip fruit": "Produce",
"other vegetables": "Produce",
"rolls/buns": "Bakery",
"pot plants": "Garden",
"beef": "Meat",
"frankfurter": "Meat",
"chicken": "Meat",
"butter": "Dairy",
"fruit/vegetable juice": "Beverages",
"packaged fruit/vegetables": "Produce",
"chocolate": "Candy",
"specialty bar": "Candy",
"butter milk": "Dairy",
"yogurt": "Dairy",
"sausage": "Meat",
"brown bread": "Bakery",
"hamburger meat": "Meat",
"root vegetables": "Produce",
"pork": "Meat",
"pastry": "Bakery",
"canned beer": "Alcohol",
"citrus fruit": "Produce",
"berries": "Produce",
"misc. beverages": "Beverages",
"coffee": "Misc",
"canned goods": "Canned Goods",
"pastry": "Bakery",
"misc. beverages": "Beverages",
"root vegetables": "Produce",
"sausage": "Meat"
}

Please note that some items like "sausage" and "pastry" appear in the grocery list more than once, but they should still be matched to the same aisle category. Also, some items like "misc. beverages" a